# Value Iteration Algorithm

Welcome to the Value Iteration assignment! This is one of the fundamental algorithms in Dynamic Programming for solving MDPs. By the end of this notebook, you'll be able to:

* Understand the Bellman Optimality Equation and how it differs from the Bellman Expectation Equation
* Implement the value update (Bellman backup) for Value Iteration
* Extract an optimal policy from the optimal value function
* Build a complete Value Iteration solver
* Understand when Value Iteration converges and why

## Dynamic Programming: The Foundation

**Dynamic Programming (DP)** requires:
- ✅ Full knowledge of the MDP (model-based)
- ✅ Can compute exact solutions
- ❌ Computationally expensive for large state spaces
- ❌ Requires environment dynamics (not always available)

## Value Iteration: Key Ideas

**Bellman Optimality Equation:**
$$V^*(s) = \max_a \sum_{s',r} p(s',r|s,a)\left[r + \gamma V^*(s')\right]$$

**Algorithm:**
1. Initialize $V(s)$ arbitrarily for all states
2. Repeat until convergence:
   - For each state $s$:
     - $V(s) \leftarrow \max_a \sum_{s',r} p(s',r|s,a)[r + \gamma V(s')]$
3. Extract policy: $\pi(s) = \arg\max_a \sum_{s',r} p(s',r|s,a)[r + \gamma V(s')]$

**Key Insight:** Value Iteration combines policy evaluation and policy improvement into a single update!

<img src="https://miro.medium.com/max/1400/1*bEhGLE8N0z_kDT6h_qjW8w.png" style="width:600px;height:300px;">

## Important Note on Submission

Please ensure:
1. No extra print statements
2. No extra code cells
3. Function parameters unchanged
4. No global variables in graded functions

## Table of Contents
- [1 - Packages](#1)
- [2 - GridWorld Environment](#2)
- [3 - Value Function Initialization](#3)
    - [Exercise 1 - initialize_value_function](#ex-1)
- [4 - Bellman Optimality Backup](#4)
    - [Exercise 2 - bellman_optimality_backup](#ex-2)
- [5 - Policy Extraction](#5)
    - [Exercise 3 - extract_policy](#ex-3)
- [6 - Value Iteration Sweep](#6)
    - [Exercise 4 - value_iteration_sweep](#ex-4)
- [7 - Complete Value Iteration](#7)
    - [Exercise 5 - value_iteration](#ex-5)
- [8 - Testing and Visualization](#8)

<a name='1'></a>
## 1 - Packages

In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib import colors
from value_iteration_tests import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)

np.random.seed(42)

<a name='2'></a>
## 2 - GridWorld Environment

We'll use FrozenLake as our test environment. It's a simple 4x4 gridworld:

```
SFFF  (S: start, F: frozen, H: hole, G: goal)
FHFH
FFFH
HFFG
```

- **States**: 16 positions (4x4 grid)
- **Actions**: 4 (left=0, down=1, right=2, up=3)
- **Rewards**: +1 for reaching goal, 0 otherwise
- **Transitions**: Deterministic (is_slippery=False)

**MDP Components:**
- $\mathcal{S}$: Set of 16 states
- $\mathcal{A}$: Set of 4 actions
- $p(s',r|s,a)$: Transition dynamics (accessible via `env.P`)
- $\gamma$: Discount factor (we'll use 0.99)

In [ ]:
# Create environment
env = gym.make('FrozenLake-v1', is_slippery=False)

n_states = env.observation_space.n
n_actions = env.action_space.n

print(f"States: {n_states}")
print(f"Actions: {n_actions}")
print(f"\nTransition dynamics available via env.P")
print(f"Example - State 0, Action 2 (right):")
print(f"env.P[0][2] = {env.P[0][2]}")
print(f"\nFormat: [(probability, next_state, reward, done)]")

<a name='3'></a>
## 3 - Value Function Initialization

The value function $V(s)$ represents the expected cumulative reward starting from state $s$ and following the optimal policy.

**Initialization strategies:**
- **Zeros**: Conservative, slower convergence
- **Random**: Can help explore different convergence paths
- **Optimistic**: Initialize with high values to encourage exploration during policy extraction

For Value Iteration, we typically initialize to zeros, except terminal states which are always 0.

<a name='ex-1'></a>
### Exercise 1 - initialize_value_function

Implement value function initialization.

In [ ]:
# GRADED FUNCTION: initialize_value_function

def initialize_value_function(n_states, terminal_states=None, init_value=0.0):
    """
    Initialize value function for all states.
    
    Arguments:
    n_states -- number of states
    terminal_states -- list of terminal state indices (value always 0)
    init_value -- initial value for non-terminal states
    
    Returns:
    V -- numpy array of shape (n_states,) with initialized values
    """
    # (approx. 3-5 lines)
    # 1. Create array of init_value for all states
    # 2. Set terminal states to 0 (if terminal_states is provided)
    # Hint: Use np.ones() or np.full() for initialization
    
    # YOUR CODE STARTS HERE
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return V

In [ ]:
# Test your implementation
V = initialize_value_function(16, terminal_states=[5, 7, 11, 12, 15], init_value=0.0)
print("Value function shape:", V.shape)
print("Value function:\n", V.reshape(4, 4))
print(f"\nTerminal states (5,7,11,12,15) should be 0: {V[[5,7,11,12,15]]}")

# Run the grader
initialize_value_function_test(initialize_value_function)

<a name='4'></a>
## 4 - Bellman Optimality Backup

The **Bellman optimality backup** is the core of Value Iteration. For a single state $s$, we compute:

$$V(s) \leftarrow \max_a \sum_{s',r} p(s',r|s,a)\left[r + \gamma V(s')\right]$$

**Breaking it down:**
1. For each action $a$ from state $s$:
   - Look up possible transitions: $p(s',r|s,a)$ (from `env.P[s][a]`)
   - For each possible next state $s'$:
     - Compute: $p \times (r + \gamma V(s'))$
   - Sum over all next states: $Q(s,a) = \sum_{s'} \ldots$
2. Take maximum over all actions: $V(s) = \max_a Q(s,a)$

**Environment dynamics format:**
```python
env.P[state][action] = [(probability, next_state, reward, done), ...]
```

<a name='ex-2'></a>
### Exercise 2 - bellman_optimality_backup

Implement the Bellman optimality backup for a single state.

In [ ]:
# GRADED FUNCTION: bellman_optimality_backup

def bellman_optimality_backup(env, V, state, gamma=0.99):
    """
    Perform Bellman optimality backup for a single state.
    
    Arguments:
    env -- Gymnasium environment with env.P (transition dynamics)
    V -- current value function, numpy array of shape (n_states,)
    state -- state to update
    gamma -- discount factor
    
    Returns:
    new_value -- updated value for this state (scalar)
    best_action -- action that achieves max value (for debugging)
    """
    # (approx. 10-12 lines)
    # 1. Initialize list to store Q-values for each action
    # 2. For each action:
    #    a. Get transitions: env.P[state][action]
    #    b. For each (prob, next_state, reward, done) in transitions:
    #       - Compute: prob * (reward + gamma * V[next_state])
    #       - Sum these values to get Q(state, action)
    # 3. Find max Q-value and corresponding action
    # 4. Return max Q-value as new_value
    
    # Hint: env.P[state][action] gives list of (probability, next_state, reward, done)
    # Hint: Use np.argmax() to find best action
    
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return new_value, best_action

In [ ]:
# Test your implementation
V_test = np.zeros(16)
V_test[15] = 1.0  # Goal state has value 1

# Test backup for state 14 (next to goal)
new_value, best_action = bellman_optimality_backup(env, V_test, state=14, gamma=0.99)
print(f"State 14 (next to goal):")
print(f"  New value: {new_value:.4f}")
print(f"  Best action: {best_action} (1=down toward goal)")
print(f"  Expected: ~0.99 (reward 0 + gamma * V[15])")

# Run the grader
bellman_optimality_backup_test(bellman_optimality_backup)

**Expected output:**
```
State 14 (next to goal):
  New value: 0.9900
  Best action: 1
```

<a name='5'></a>
## 5 - Policy Extraction

Once we have the optimal value function $V^*$, we can extract the optimal policy:

$$\pi^*(s) = \arg\max_a \sum_{s',r} p(s',r|s,a)\left[r + \gamma V^*(s')\right]$$

This is similar to the Bellman backup, but instead of taking the **max value**, we take the **argmax action**.

**Policy types:**
- **Deterministic**: $\pi(s)$ returns a single action
- **Stochastic**: $\pi(a|s)$ returns probability distribution over actions

We'll implement a deterministic policy.

<a name='ex-3'></a>
### Exercise 3 - extract_policy

Extract the greedy policy from a value function.

In [ ]:
# GRADED FUNCTION: extract_policy

def extract_policy(env, V, gamma=0.99):
    """
    Extract greedy policy from value function.
    
    Arguments:
    env -- Gymnasium environment
    V -- value function, numpy array of shape (n_states,)
    gamma -- discount factor
    
    Returns:
    policy -- numpy array of shape (n_states,) with best action for each state
    """
    # (approx. 8-10 lines)
    # 1. Initialize policy array
    # 2. For each state:
    #    a. Compute Q(s,a) for all actions (similar to bellman_backup)
    #    b. Select action with highest Q-value
    #    c. Store in policy array
    # 
    # Hint: You can reuse logic from bellman_optimality_backup
    # Hint: For each action, compute sum over transitions
    
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return policy

In [ ]:
# Test your implementation
V_test = np.zeros(16)
V_test[15] = 1.0

policy = extract_policy(env, V_test, gamma=0.99)
print("Policy shape:", policy.shape)
print("Policy (as 4x4 grid):")
print(policy.reshape(4, 4))
print("\nAction mapping: 0=left, 1=down, 2=right, 3=up")
print("\nState 14 should choose action 2 (right toward goal)")
print(f"Policy[14] = {policy[14]}")

# Run the grader
extract_policy_test(extract_policy)

<a name='6'></a>
## 6 - Value Iteration Sweep

A **sweep** means updating the value function for **all states** once. 

In Value Iteration, we have two update strategies:

**1. Synchronous (two arrays):**
```python
V_new = np.copy(V)
for s in all_states:
    V_new[s] = max_a sum(...)
V = V_new
```

**2. Asynchronous (in-place):**
```python
for s in all_states:
    V[s] = max_a sum(...)  # Update immediately
```

Asynchronous often converges faster! We'll implement the in-place version.

**Convergence check:**
- Measure max change: $\delta = \max_s |V_{new}(s) - V_{old}(s)|$
- Stop when $\delta < \theta$ (threshold)

<a name='ex-4'></a>
### Exercise 4 - value_iteration_sweep

Perform one complete sweep over all states.

In [ ]:
# GRADED FUNCTION: value_iteration_sweep

def value_iteration_sweep(env, V, gamma=0.99):
    """
    Perform one sweep of value iteration (update all states).
    
    Arguments:
    env -- Gymnasium environment
    V -- current value function, numpy array of shape (n_states,)
    gamma -- discount factor
    
    Returns:
    V -- updated value function
    delta -- maximum change in value function (for convergence check)
    """
    # (approx. 6-8 lines)
    # 1. Initialize delta = 0
    # 2. For each state:
    #    a. Store old value: v_old = V[state]
    #    b. Compute new value using bellman_optimality_backup
    #    c. Update V[state] = new_value (in-place!)
    #    d. Update delta = max(delta, |new_value - v_old|)
    # 3. Return updated V and delta
    
    # Hint: Use your bellman_optimality_backup function
    # Hint: delta tracks the largest change across all states
    
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return V, delta

In [ ]:
# Test your implementation
V_test = initialize_value_function(16, terminal_states=[5,7,11,12,15])

print("Initial V (first row):", V_test[:4])

V_updated, delta = value_iteration_sweep(env, V_test, gamma=0.99)
print(f"After 1 sweep:")
print(f"  V (first row): {V_updated[:4]}")
print(f"  Delta: {delta:.6f}")

# Run a few more sweeps
for i in range(3):
    V_updated, delta = value_iteration_sweep(env, V_updated, gamma=0.99)
    print(f"After {i+2} sweeps: delta = {delta:.6f}")

# Run the grader
value_iteration_sweep_test(value_iteration_sweep)

<a name='7'></a>
## 7 - Complete Value Iteration

Now we put it all together! The complete Value Iteration algorithm:

```
Initialize V(s) for all s
repeat:
    Δ ← 0
    for each state s:
        v ← V(s)
        V(s) ← max_a Σ p(s',r|s,a)[r + γV(s')]
        Δ ← max(Δ, |v - V(s)|)
until Δ < θ
```

**Convergence:**
- Value Iteration is guaranteed to converge to $V^*$
- Convergence is geometric: error decreases by factor of $\gamma$ each sweep
- Typical threshold: $\theta = 10^{-6}$ to $10^{-4}$

<a name='ex-5'></a>
### Exercise 5 - value_iteration

Implement the complete Value Iteration algorithm.

In [ ]:
# GRADED FUNCTION: value_iteration

def value_iteration(env, gamma=0.99, theta=1e-6, max_iterations=1000):
    """
    Value Iteration algorithm.
    
    Arguments:
    env -- Gymnasium environment
    gamma -- discount factor
    theta -- convergence threshold
    max_iterations -- maximum number of iterations
    
    Returns:
    V -- optimal value function
    policy -- optimal policy
    n_iterations -- number of iterations until convergence
    deltas -- list of delta values (for plotting convergence)
    """
    # (approx. 12-15 lines)
    # 1. Initialize value function (hint: use your initialize_value_function)
    #    FrozenLake holes: [5,7,11,12], goal: [15]
    # 2. Initialize deltas list to track convergence
    # 3. Loop until convergence or max_iterations:
    #    a. Perform one sweep (hint: use your value_iteration_sweep)
    #    b. Store delta in deltas list
    #    c. If delta < theta: break (converged!)
    # 4. Extract optimal policy (hint: use your extract_policy)
    # 5. Return V, policy, n_iterations, deltas
    
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return V, policy, n_iterations, deltas

<a name='8'></a>
## 8 - Testing and Visualization

Let's run the complete Value Iteration algorithm and visualize the results!

In [ ]:
# Run Value Iteration
print("Running Value Iteration on FrozenLake...\n")
V_optimal, policy_optimal, n_iter, deltas = value_iteration(
    env, gamma=0.99, theta=1e-6, max_iterations=1000
)

print(f"Converged in {n_iter} iterations\n")
print("Optimal Value Function:")
print(V_optimal.reshape(4, 4))
print("\nOptimal Policy (0=L, 1=D, 2=R, 3=U):")
print(policy_optimal.reshape(4, 4))

In [ ]:
# Visualize convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Delta over iterations
ax1.plot(deltas, linewidth=2)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Delta (max value change)')
ax1.set_title('Value Iteration Convergence')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Value function heatmap
V_grid = V_optimal.reshape(4, 4)
im = ax2.imshow(V_grid, cmap='viridis')
ax2.set_title('Optimal Value Function')
ax2.set_xticks(range(4))
ax2.set_yticks(range(4))

# Add value annotations
for i in range(4):
    for j in range(4):
        text = ax2.text(j, i, f'{V_grid[i, j]:.2f}',
                       ha="center", va="center", color="white", fontsize=10)

plt.colorbar(im, ax=ax2)
plt.tight_layout()
plt.show()

print(f"\nKey Observations:")
print(f"1. Delta decreases geometrically (linear on log scale)")
print(f"2. Values are highest near the goal (state 15)")
print(f"3. Hole states (5,7,11,12) have value 0")

In [ ]:
# Visualize policy with arrows
def plot_policy(policy, title="Optimal Policy"):
    """Plot policy as arrows on grid."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create grid
    grid = np.zeros((4, 4))
    ax.imshow(grid, cmap='Blues', alpha=0.3)
    
    # Arrow directions
    arrows = {0: '←', 1: '↓', 2: '→', 3: '↑'}
    
    # Plot arrows
    for state in range(16):
        if state in [5, 7, 11, 12]:  # Holes
            continue
        i, j = state // 4, state % 4
        action = policy[state]
        ax.text(j, i, arrows[action], ha='center', va='center', 
                fontsize=30, color='darkblue')
    
    # Mark special states
    ax.text(0, 0, 'S', ha='left', va='top', fontsize=12, color='green', weight='bold')
    ax.text(3, 3, 'G', ha='right', va='bottom', fontsize=12, color='red', weight='bold')
    
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.set_title(title, fontsize=16)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_policy(policy_optimal, "Optimal Policy from Value Iteration")

In [ ]:
# Test the policy
def evaluate_policy(env, policy, n_episodes=100):
    """Evaluate policy by running episodes."""
    successes = 0
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        done = False
        
        while not done:
            action = int(policy[state])
            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            if reward == 1:
                successes += 1
                break
    
    return successes / n_episodes

success_rate = evaluate_policy(env, policy_optimal, n_episodes=100)
print(f"\nPolicy Evaluation (100 episodes):")
print(f"Success rate: {success_rate * 100:.1f}%")
print(f"\nFor deterministic FrozenLake, optimal policy should reach 100%!")

env.close()

## Congratulations!

You've successfully implemented Value Iteration from scratch! Here's what you've learned:

✅ How to initialize value functions properly

✅ The Bellman optimality backup equation

✅ How to extract greedy policies from value functions

✅ The complete Value Iteration algorithm

✅ How to check for convergence

### Key Takeaways:

1. **Value Iteration** combines policy evaluation and improvement into one update
2. **Convergence** is guaranteed and geometric (fast!)
3. **Model-based**: Requires full knowledge of MDP dynamics
4. **Optimal**: Finds the true optimal policy for the MDP
5. **In-place updates** often converge faster than synchronous

### Value Iteration vs Q-Learning:

| Aspect | Value Iteration | Q-Learning |
|--------|----------------|------------|
| Knowledge | Requires model | Model-free |
| Updates | All states per iteration | One state per step |
| Convergence | Guaranteed, fast | Guaranteed (with conditions) |
| Scalability | Poor (large state spaces) | Better (sampling) |
| Optimality | Exact optimal | Converges to optimal |

### Next Steps:

- Learn about **Policy Iteration** (evaluates full policy before improvement)
- Compare convergence speed: Policy Iteration vs Value Iteration
- See how **Monte Carlo** methods work without a model
- Understand why **Q-Learning** is preferred for large/unknown MDPs